# Running Parallel OSDT

In [3]:
# third-party imports
from time import time

# local imports
from lib.parallel_osdt_classifier import ParallelOSDTClassifier
from lib.data_processing import read_dataset

# Using COMPAS as an example
dataset = read_dataset('data/preprocessed/compas-binary.csv') 
(n, m) = dataset.shape
X = dataset.values[:n,:m-1]
y = dataset.values[:n,-1]

hyperparameters = {
    'regularization': 0.005, # Regularization coefficient which effects the penalty on model complexity

    'max_depth': float('Inf'), # User-specified limit on the model
    'max_time': 60, # User-specified limit on the runtime 

    'workers': 1, # Parameter that varies based on how much computational resource is available

    'visualize': True, # Toggle whether a rule-list visualization is rendered
    'verbose': False, # Toggle whether event messages are printed
    'log': False, # Toggle whether client processes log to logs/work_<id>.log files
    'profile': False, # Toggle Snapshots for Profiling Memory Usage
    
    'configuration': { # More configurations around toggling optimizations and prioritization options
        'priority_metric': 'uniform', # Decides how tasks are prioritized
        'deprioritization': 0.01, # Decides how much to push back a task if it has pending dependencies

        # Note that Leaf Permutation Bound (Theorem 6) is 
        # Toggles the assumption about objective independence when composing subtrees (Theorem 1)
        # Disabling this actually breaks convergence due to information loss
        'hierarchical_lowerbound': True, 
        # Toggles whether problems are pruned based on insufficient accuracy (compared to other results) (Lemma 2)
        'look_ahead': True,
        # Toggles whether a split is avoided based on insufficient support (proxy for accuracy gain) (Theorem 3)
        'support_lowerbound': True,
        # Toggles whether a split is avoided based on insufficient potential accuracy gain (Theorem 4)
        'incremental_accuracy_lowerbound': True,
        # Toggles whether a problem is pruned based on insufficient accuracy (in general) (Theorem 5)
        'accuracy_lowerbound': True,
        # Toggles whether problem equivalence is based solely on the capture set (Similar to Corollary 6)
        'capture_equivalence': True,
        # Hamming distance used to propagate bounding information of similar problems (Theorem 7 + some more...)
        "similarity_threshold": 0,
        # Toggles whether equivalent points contribute to the lowerbound (Proposition 8 and Theorem 9)
        'equivalent_point_lowerbound': True,

        # Toggles compression of dataset based on equivalent point aggregation
        'equivalent_point_compression': True,
        # Toggles whether asynchronous tasks can be cancelled after being issued
        'task_cancellation': True,
        # Toggles whether look_ahead prunes using objective upperbounds (This builds on top of look_ahead)
        'interval_look_ahead': True,
        # Cooldown timer (seconds) on synchornization operations
        'synchronization_cooldown': 0.01,
        # Cache Limit
        'cache_limit': float('Inf')
    }
}

start = time()
model = ParallelOSDTClassifier(**hyperparameters)
model.fit(X, y)
print('Runtime: {} Seconds'.format(time() - start))
print('Prediction: \n{}'.format(model.predict(X)))
print('Training Accuracy: {}'.format(model.score(X, y)))
print('Visualization: \n{}'.format(model.model.visualization))

Runtime: 8.963286876678467 Seconds
Prediction: 
[[0]
 [0]
 [1]
 ...
 [0]
 [0]
 [1]]
Training Accuracy: 0.669031417402635
Visualization: 
(_,_,_,0,_,_,_,_,_,_,_,0) => 0 (Risk Contribution = 0.12878746199507746)
(_,_,_,1,_,_,_,0,_,_,_,0) => 1 (Risk Contribution = 0.022518459533806285)
(_,_,_,1,_,_,_,1,_,_,0,0) => 0 (Risk Contribution = 0.0708751990734038)
(_,_,_,1,_,_,_,1,_,_,1,0) => 1 (Risk Contribution = 0.022228898219197917)
(_,_,_,_,_,_,_,_,_,_,_,1) => 1 (Risk Contribution = 0.11155856377587954)


# Running Sequential OSDT

In [4]:
# All dependencies of this notebook

# third-party imports
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

# local imports
from lib.osdt_classifier import OSDTClassifier
from lib.model_selection import train_cross_validate
from lib.data_processing import read_dataset

# Using COMPAS as an example
dataset = read_dataset('data/preprocessed/compas-binary.csv') 
(n, m) = dataset.shape
X = dataset.values[:,:-1]
y = dataset.values[:,-1]

hyperparameters = {
    'regularization': 0.005, # Regularization coefficient which effects the penalty on model complexity
    'max_depth': float('Inf'), # User-specified limit on the model
    'max_time': float('Inf'), # User-specified limit on the runtime 
    
    'configuration': { # More configurations around toggling optimizations and prioritization options
        'priority_metric': 'curiosity',
        'look_ahead': True,
        'support_lowerbound': True,
        'incremental_accuracy_lowerbound': True,
        'accuracy_lowerbound': True,
        'equivalent_point_lowerbound': True,
    }
}

model = OSDTClassifier(**hyperparameters)
model.fit(X, y)
print('Runtime: {}'.format(time() - start))
print('Prediction: \n{}'.format(model.predict(X)))
print('Training Accuracy: {}'.format(model.score(X, y)))

nrule: 12
ndata: 6907
gr: [0.00438766 0.0037798  0.00481064 0.00654486 0.00818606 0.00697492
 0.00861952 0.01958141 0.0261244  0.00328364 0.0004441  0.03628272]
order: [11, 8, 7, 6, 4, 5, 3, 2, 0, 1, 9, 10]
odr: [11, 8, 7, 6, 4, 5, 3, 2, 0, 1, 9, 10]
the order of x's columns:  [11, 8, 7, 6, 4, 5, 3, 2, 0, 1, 9, 10]
>>> log: False
>>> support bound: True
>>> accu_support: True
>>> accurate support bound: True
>>> equiv points bound: True
>>> lookahead bound: True
prior_metric= curiosity
COUNT_UNIQLEAVES: 3252
COUNT_LEAFLOOKUPS: 126103
total time:  13.383165121078491
lambda:  0.005
leaves:  [(1,), (-7, -1), (-1, 7, 12), (-12, -3, -1, 7), (-12, -1, 3, 7)]
num_captured:  [2174, 2898, 483, 205, 1147]
num_captured_incorrect:  [736, 855, 164, 76, 455]
prediction:  [1, 0, 1, 1, 0]
Objective:  0.35596858259736497
Accuracy:  0.669031417402635
COUNT of the best tree:  18195
time when the best tree is achieved:  1.0074620246887207
TOTAL COUNT:  241306
best_is_cart False
Runtime: 23.080535888671875

# Sample Experiment